In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm.notebook import tqdm as tqdm
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec


class People:
    pos=np.array([])
    vel=np.array([])
    healthycolor='lightsteelblue'
    sickcolor='red'
    recorveredcolor='seagreen'
    color=healthycolor
    velwait=1.0
    
    h=0.03
    timer=0
    
    def __init__(self,velwait=1.0):
        self.color='lightsteelblue'
        self.velwait=velwait
        
        self.pos=np.random.rand(2)
        self.vel=self.velwait*2*(np.random.rand(2)-0.5)
        
    def set_sick(self):
        self.color='red'
    
    def move(self):
        
        pos=self.pos
        self.pos=self.pos+self.vel*self.h
        
        while not (0<self.pos[0]<1 and 0<self.pos[1]<1):
            self.vel=self.velwait*2*(np.random.rand(2)-0.5)
            self.pos=pos+self.vel*self.h
            
        if self.color==self.sickcolor:
            self.timer+=1
            if self.timer>40:
                self.color=self.recorveredcolor

        
    def check_range(self, p):
        r=np.sqrt(((self.pos-p.pos)**2).sum())
        #print(r)
        if r<self.h:
            self.vel=self.velwait*2*(np.random.rand(2)-0.5)

            if (self.color==self.sickcolor or p.color==self.sickcolor):
                if self.color==self.healthycolor:
                    self.color=self.sickcolor
                if p.color==self.healthycolor:
                    p.color=self.sickcolor

### Main ###

p=[]
#粒子の数
N=100

#シミュレーション回数
Simtimes=150

#不活発粒子のパーセンテージ
R=70

sickflag=True
#粒子を作る
for i in range(N):
    if np.random.rand()*100<=R:
        p.append(People(velwait=0.01))
    else:
        p.append(People())
        if sickflag==True:
            #感染粒子を一つ入れる
            p[i].set_sick()
            sickflag=False



fig = plt.figure(figsize=(8,12), tight_layout=True)
gs = fig.add_gridspec(3, 1)
ax1 = fig.add_subplot(gs[0:2, 0])
ax2 = fig.add_subplot(gs[2, 0])

Helethy=np.zeros(Simtimes)
Sick=np.zeros(Simtimes)
Recorvered=np.zeros(Simtimes)
Index=np.arange(Simtimes)

for i in tqdm(range(Simtimes)):
    ax1.set_xlim(0,1)
    ax1.set_ylim(0,1)
    sick=0
    recorvered=0
    
    for j in range(N):
        p[j].move()
        
    for j in range(N):
        for m in range(j+1,N):
            p[j].check_range(p[m])
        ax1.plot(p[j].pos[0], p[j].pos[1],'o',ms=15, color=p[j].color)
        
        if p[j].color==p[j].sickcolor:
            sick+=1
        elif p[j].color==p[j].recorveredcolor:
            recorvered+=1
        
        healthy=N-sick-recorvered
    
    Helethy[i]=healthy
    Sick[i]=sick
    Recorvered[i]=recorvered

    
    ax2.bar(Index, Sick,width=1,color=People.sickcolor)
    ax2.bar(Index, Helethy, bottom=Sick, width=1,color=p[0].healthycolor)
    ax2.bar(Index, Recorvered, bottom=Helethy+Sick ,width=1, color=p[0].recorveredcolor)
    
    plt.savefig('coloanim/colo{:04d}.png'.format(i))
    ax1.cla()
    ax2.cla()
    



In [ ]:
#GIF アニメにする
from PIL import Image
import glob

files = sorted(glob.glob('./coloanim/*.png'))  
images = list(map(lambda file : Image.open(file) , files))
images[0].save('./image100_150_90p.gif' , save_all = True , append_images = images[1:] , duration = 100 , loop = 0)